In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime , timedelta
import re
import copy
from sqlalchemy import create_engine
import calendar

In [2]:
conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='123',
                                                                     host='localhost',
                                                                     port='5432',
                                                                     db='postgres'
                                                                     )
engine = create_engine(conn_string, paramstyle='format', isolation_level="READ UNCOMMITTED")

In [3]:
path = 'D:/AS/'
state = 'AS'

In [4]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [5]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']==state]
slab_new = slab_new[slab_new['vh_class'].notnull()]

In [6]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')
slab_new.reset_index(drop=True, inplace=True)

In [7]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')
slab_add_new = slab_add_new[slab_add_new['state_cd']==state]

In [8]:
vt_tax_based_on = pd.read_csv(path+'/vt_tax_based_on.csv')
vt_owner = pd.read_csv(path+'/vt_owner.csv')
vt_owner = vt_owner[vt_owner['State Cd']==state]
vt_permit = pd.read_csv(path+'/vt_permit.csv')
tm_office = pd.read_csv('D:/RJ/tm_office.csv')
tm_office = tm_office[tm_office['state_cd']==state]

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
def_list = pd.read_csv(r'D:\AS\New folder\Defaulters List (Kamrup) v4.csv')
def_list = def_list[def_list['Vh Class'].notnull()]
#def_list['Vh Class'].value_counts()
#def_list['Vh Class'].unique()
#def_list = def_list.sample(frac=0.01,replace=False,random_state=1)
def_list.reset_index(drop=True, inplace=True)
def_list.shape

(7625, 20)

In [12]:
def calculate_rates(true_df,params_info):
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print('t_params',t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        rate = pd.read_sql_query(repr('select '+r_formula)[1:-1],con=engine)
        rate = rate.loc[0,'?column?']
        #print(rate)
        rates.append(rate)        
    true_df['Rate']=rates
    return true_df

In [11]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if '-' in v:
                        v = v.replace('-','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
    return c_formula

In [10]:
def calculate_formula(formula_df,params_info):
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        #print('l',l)
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_form_convert(cf_params,params_info, condition_formula)
        #print(c_formula)
        
        # To be tested
        for m in cf_params:
            if m != '<>':
                val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])                
                c_formula = c_formula.replace(str(m),val)
                #print(c_formula,str(m),val)
        c_formula = c_formula.replace(repr('nan'),'-100')
        #print(c_formula)
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R' and 'nan' not in c_formula.lower():
            cf_result = pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)
            cf_result = cf_result.loc[0,'?column?']
        else:
            cf_result = False
        if cf_result==True:
            #print('Ind',ind,l)
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    return [ind,summary_df]

In [13]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print('TBO shape',tbo_df.shape)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%d-%m-%Y'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [68]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,rcpt_dt,tax_mode):
    for pi in range(len(param_info_owner)):
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        p_value = ''

        # Only for Assam
        if tab=='own': 
            p_date = 'Purchase Dt'
        else:
            p_date= 'purchase_dt'
        purchase_dt = ot_df[p_date].to_list()[0]
        p_valid_date = purchase_dt[:2].isdigit()
        
        if p_valid_date==False and tab=='tbo':
            #print(p_valid_date,tab)
            purchase_dt = vt_owner.loc[vt_owner['Regn No']==regn_no][p_date].to_list()[0]
            
        #Any format of purchase date converted to date format
        p_valid_date = purchase_dt[:2].isdigit()
        if purchase_dt[:4].isdigit():
            purchase_dt =datetime.strptime(purchase_dt,'%Y-%m-%d')
        elif purchase_dt[3:6].isalpha() :
            purchase_dt =datetime.strptime(purchase_dt,'%d-%b-%Y')
        elif p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%m-%Y')
            
        #Parameter Extraction    
        if par=='vch_age' or par=='fin_vch_age':
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='vch_age_months':
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age':
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        elif par=='payment_date':
            p_value = rcpt_dt
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['Vh Class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = 'N'
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='permit_sub_catg':
            if 'pmt_catg' in ot_df.columns:
                p_value = ot_df['pmt_catg'].to_list()[0]
            else:
                p_value = float("nan")
        elif par=='delay_days':
            p_value = rcpt_dt-tax_from
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
        elif par == 'delay_months':
            p_value = relativedelta(rcpt_dt, tax_from)
            if p_value.days>0:
                bal_m = 1
            p_value = p_value.years * 12 + p_value.months + bal_m
        else:
            
            if tab=='own':
                col= col.replace('_',' ').title() #Only for Assam
            p_value= ot_df[col].to_list()[0]
        #print(col,' ',par,' ',code,' ',p_value)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
    return params_info

In [177]:
def extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,pur_cd,tax_mode,final_rate):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['Regn No']==regn_no]
        owner_df= copy.deepcopy(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            #for j in c :
                #p= j.replace('_',' ').title()
                #tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]
    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        tab = 'tbo'
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,rcpt_dt,tax_mode)
    else:
        tab='own'
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,rcpt_dt,tax_mode)
    #print('Shape',params_info)
    if len(param_info_other)>0:  
        for pi in range(len(param_info_other)):
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            if pd.isnull(params_info.loc[params_info['code']==code]['P_value']).to_list()[0]==True:
                if par == 'domain_cd':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Domain Cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Domain Cd Available')
                elif par == 'permit_type':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Pmt Type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    else:
                        p_value = None
                elif par== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                elif par=='tax_amt':
                    p_value = final_rate
                elif par=='tax_period_sr_no':
                    p_value = 1
                elif par=='service_type':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Service Type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                #print('Other',code,par)
                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
                p_value = None
    return params_info

In [168]:
date_format = "%d-%b-%Y"
df = pd.DataFrame({'a':[]})
def calculate_tax(regn_no,vh_class,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Y' ,state_cd=state,off_cd=141,tm='n',result=df):
    rcpt_dt = tax_upto
    
    #print(vh_class, pur_cd, state_cd,tax_mode, tax_from)
    slab_new2 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_from)]
    #print(slab_new2)
    tax_modes = list(set(slab_new2['tax_mode'].to_list()))

    if len(tax_modes) ==0:
        print('No Slab Found')
    else:
        for tax_mode in tax_modes:
            slab_new1 = slab_new2[slab_new2['tax_mode']==tax_mode]
            #print(tax_mode,slab_new1)
            slab_codes = slab_new1['slab_code'].to_list()
            #print(pur_cd,tax_mode,'Slab Codes Applicable:',slab_codes)
            formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
            formula_df = formula_df[formula_df['condition_formula']!='0=0']

            #Identify the parameters required for calculating the formulas
            params = []
            for j in range(len(formula_df)):
                formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
                for k in re.split(r'[()\*\s]\s*',formula):
                    if '<' in k and '>' in k:
                        params.append(k[k.find('<'):(k.find('>')+1)])
            params = list(set(params))
            #print('Required Parameters :',params)
            params_info = slab_fields.loc[slab_fields['code'].isin(params)]
            params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
            param_info_owner = params_info[params_info['def_table']=='vt_owner']
            #print('Owner',param_info_owner['column - slab_field'].to_list())
            param_info_other = params_info[params_info['def_table']!='vt_owner']
            #print('Other',param_info_other['column - slab_field'].to_list())
            param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
            #print('TBO',param_info_tbo['column - slab_field'].to_list())
            final_rate = result.loc[0,tax_mode+'_rate']
            #print(tax_mode,final_rate)
            params_info = extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,pur_cd,tax_mode,final_rate)
            #print('PARAMS INFO',tax_mode,'\n',params_info)
            ind = calculate_formula(formula_df,params_info)[0]
            summary_df = calculate_formula(formula_df,params_info)[1]
            
            #if pur_cd==57:
            #print(summary_df)
            if len(ind)==0:
                #print('No True Condition')
                continue
            #Module for calculation of rates for true conditions
            true_df = formula_df.loc[ind,:]
            true_df.reset_index(drop=True, inplace=True)
            #print('True_df',true_df)
            #print(t_params)

            true_df_t = true_df[(true_df['type_flag']=='T') | (true_df['type_flag']=='M') | (true_df['type_flag']=='X') ]
            true_df_r = true_df[true_df['type_flag']=='R']
            true_df_s = true_df[true_df['type_flag']=='S']

            true_df_t = calculate_rates(true_df_t,params_info)
            #print('T',true_df_t)

            t_df = true_df_t[(true_df_t['type_flag']=='T')]
            t_df.reset_index(drop=True, inplace=True)               
            #print(t_df)
            if len(t_df)>0:
                #t_rate = true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]
                #print('t_rate',t_rate)
                #if len(t_df)>1:
                t_rate = 0
                upt = datetime.strptime('01-Jan-1900','%d-%b-%Y')
                #until = max(slab_new1[slab_new1['slab_code'] in t_df['slab_code'].to_list() ]['date_to'].to_list())
                for t in range(len(t_df)):
                    sc = t_df.loc[t,'slab_code']
                    f_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_from']
                    t_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_to']
                    fro = max(f_dt.item(),tax_from,upt)   
                    upt = min(t_dt.item(),tax_upto)
                    n_days = (upt - fro).days + 1
                    n_quart = math.ceil(n_days/92) +1
                    if tax_mode=='Q':
                        per_day_rate = t_df.loc[t,'Rate'] / 92
                        until = tax_from + timedelta(n_quart*92)
                    elif tax_mode=='Y5':
                        per_day_rate = t_df.loc[t,'Rate'] / (365*5)
                        until = tax_from + timedelta(365*5)
                    elif tax_mode=='Y10':
                        per_day_rate = t_df.loc[t,'Rate'] / (365*10)
                        until = tax_from + timedelta(365*10)
                    elif tax_mode=='Y15':
                        per_day_rate = t_df.loc[t,'Rate'] / (365*15)
                        until = tax_from + timedelta(365*15)
                    else:
                        per_day_rate = t_df.loc[t,'Rate'] / 365
                        until = tax_from + timedelta(365)
                    t_rate = t_rate + per_day_rate *n_days
                    #print(pur_cd,t_rate,fro,upt,t_df.loc[t,'Rate'],per_day_rate,n_days,tm)
            #Module for dealing Minimum M and Amount T 
            if len(set(true_df_t['condition_formula'].to_list()))>0:
                #true_df_t = true_df_t[true_df_t['condition_formula']!='1=1']
                if len({'T','M'} - set(true_df_t['type_flag'].to_list()))==0:
                    if true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]> true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()[0]:
                        final_rate = t_rate
                    else:
                        final_rate = true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()
                elif len({'T','X'} - set(true_df_t['type_flag'].to_list()))==0:
                    #print('Here')
                    final_rate = true_df_t[true_df_t['type_flag']=='X']['Rate'].to_list()
                else:
                    final_rate = t_rate
            #else:       
            #    final_rate = true_df['Rate'].to_list()
            #else:
            #    final_rate = 0

            #tax_mode== 'Q'
            #Module for Surcharge
            if len(true_df_s) > 0 :
                params_info.loc[params_info['code'] == '<58>','P_value']=final_rate
                true_df_s = calculate_rates(true_df_s,params_info)
                #print(true_df_s)
                surcharge = true_df_s[true_df_s['type_flag']=='S']['Rate'].to_list()
                #print(final_rate,surcharge)
                final_rate = final_rate + surcharge[0]
            #print(final_rate)
            #final_rates.append(final_rate)
            #print(pur_cd,tax_mode,round(final_rate),until)
            if pur_cd ==58:
                if tax_mode == 'Q':
                    result.loc[result['regn_no']==regn_no,'Q_rate'] = final_rate
                    result.loc[result['regn_no']==regn_no,'Q_upto'] = until
                elif tax_mode == 'Y':
                    result.loc[result['regn_no']==regn_no,'Y_rate'] = final_rate
                    result.loc[result['regn_no']==regn_no,'Y_upto'] = until
                elif tax_mode == 'Y5':
                    result.loc[result['regn_no']==regn_no,'Y5_rate'] = final_rate
                    result.loc[result['regn_no']==regn_no,'Y5_upto'] = until
                elif tax_mode == 'Y10':
                    result.loc[result['regn_no']==regn_no,'Y10_rate'] = final_rate
                    result.loc[result['regn_no']==regn_no,'Y10_upto'] = until
                elif tax_mode == 'Y15':
                    result.loc[result['regn_no']==regn_no,'Y15_rate'] = final_rate
                    result.loc[result['regn_no']==regn_no,'Y15_upto'] = until
                #else:
                    #print('Tax mode ',tax_mode,' Rate ',final_rate)
            else:
                if tax_mode == 'Q':
                    result.loc[result['regn_no']==regn_no,'Q_fine'] = final_rate
                elif tax_mode == 'Y':
                    result.loc[result['regn_no']==regn_no,'Y_fine'] = final_rate
                elif tax_mode == 'Y5':
                    result.loc[result['regn_no']==regn_no,'Y5_fine'] = final_rate
                elif tax_mode == 'Y10':
                    result.loc[result['regn_no']==regn_no,'Y10_fine'] = final_rate
                elif tax_mode == 'Y15':
                    result.loc[result['regn_no']==regn_no,'Y15_fine'] = final_rate   
    return result

In [178]:
nc_regn_no_list = []
result_df = pd.DataFrame({'regn_no': [],  'vh_class': [],'latest_tax_upto': [],'Q_rate':[],'Q_fine':[],'Q_upto':[],'Y_rate':[],'Y_fine':[],'Y_upto':[],
                          'Y5_rate':[],'Y5_fine':[],'Y5_upto':[],'Y10_rate':[],'Y10_fine':[],'Y10_upto':[],'Y15_rate':[],'Y15_fine':[],'Y15_upto':[],
                          })
for i in range(7624,7625):  
    print('case',i)
    regn_no = def_list['Regn No'].to_list()[i]    
    state_cd = state
    #print(regn_no,state)
    vh_class = def_list['Vh Class'].to_list()[i]
    tax_from = def_list['Max Upto'].to_list()[i]
    #print(tax_from)
    tax_from = datetime.strptime(tax_from, '%d-%m-%Y')
    tax_upto = datetime.strptime('03-Oct-2019','%d-%b-%Y')
    #print('Delayed',tax_upto-tax_from)
    result = pd.DataFrame({'regn_no': [regn_no], 
                           'vh_class': [vh_class],
                           'latest_tax_upto': [tax_from],
                           'Q_rate':[0],
                           'Q_fine':[0],
                           'Q_upto':[0],
                           'Y_rate':[0],
                           'Y_fine':[0],
                           'Y_upto':[0],
                           'Y5_rate':[0],
                           'Y5_fine':[0],
                           'Y5_upto':[0],
                           'Y10_rate':[0],
                           'Y10_fine':[0],
                           'Y10_upto':[0],
                           'Y15_rate':[0],
                           'Y15_fine':[0],
                           'Y15_upto':[0],
                          })
    result = calculate_tax(regn_no,vh_class,tax_from,tax_upto,state_cd=state,result=result)
    #print('AMT RESULT\n',result)
    
    fine_result = calculate_tax(regn_no,vh_class,tax_from,tax_upto, pur_cd = 57,state_cd=state,result=result) 
    #print(tax_fine)
    #if tax_amt!=c_tax_amt: 
    #    print('Mismatch',i,regn_no, vh_class,str(tax_from)[:11],str(tax_upto)[:11],tax_amt,c_tax_amt,tax_modes,untils,c_tax_fine,tax_fine)
    #else:
    #    print(i, regn_no,vh_class,str(tax_from)[:11],str(tax_upto)[:11], tax_amt, c_tax_amt,c_tax_fine,tax_fine)
    #summary_df = c_tax_amt[1]
    #print('Report',regn_no,tax_amt,tax_fine,str(tax_from)[:11],str(tax_upto)[:11])
    print(result)
    
    #result_df = pd.merge(def_df,def_list,on=['Regn No','Off Cd','Vh Class'])
#result

case 7624
     regn_no  vh_class latest_tax_upto       Q_rate  Q_fine  \
0  AS01J1817      57.0      2008-03-31  18278.26087       0   

                Q_upto        Y_rate         Y_fine               Y_upto  \
0  2020-02-01 00:00:00  17276.712329  242046.739726  2009-03-31 00:00:00   

   Y5_rate  Y5_fine  Y5_upto  Y10_rate  Y10_fine  Y10_upto  Y15_rate  \
0        0        0        0         0         0         0         0   

   Y15_fine  Y15_upto  
0         0         0  


In [138]:
def_list.tail()

,Max Upto,Eng No,Fuel,Imported Vch,Ld Wt,Maker,Maker Model,Off Cd,Owner Cd,Purchase Dt,Regn Dt,Regn No-1,Seat Cap,Sleeper Cap,Stand Cap,Status,Unld Wt,Vch Catg,Vh Class,Regn No
7620,31-12-2015,E483CDAB224456,2.0,N,7200.0,92.0,1059XP/3350,1.0,1.0,29-04-2010,07-05-2010,AS01CC5489,2.0,NaN,NaN,A,2800.0,OTH,59.0,AS01CC5489
7621,30-09-2019,497SPTC43NWY658984,2.0,N,2800.0,86.0,SUMO GOLD LX,1.0,1.0,29-03-2014,02-04-2014,AS01FC4105,9.0,0.0,0.0,A,1820.0,LPV,71.0,AS01FC4105
7622,30-09-2019,R5H2680609,2.0,N,975.0,71.0,APE D600,1.0,1.0,28-09-2015,28-09-2015,AS01GC4650,2.0,0.0,0.0,A,435.0,3WT,58.0,AS01GC4650
7623,30-09-2019,497TC73PTY845859,2.0,N,7700.0,86.0,55062042000R,1.0,1.0,20-01-2017,02-03-2017,AS01HC6220,34.0,0.0,0.0,Y,2910.0,HPV,73.0,AS01HC6220
7624,31-03-2008,AEMBFJ78632,1.0,N,550.0,4663.0,BAJAJ 3 WHEELER(AUTORICKSHAW),1.0,1.0,16-03-2000,16-03-2000,AS01J1817,3.0,NaN,NaN,A,300.0,OTH,57.0,AS01J1817


In [172]:
vt_permit[vt_permit['Regn No']=='AS01HC6220']['Service Type']

2145    6.0
Name: Service Type, dtype: float64

In [174]:
vt_tax_based_on.columns

Index(['Number of Records', 'ac_fitted', 'audio_fitted', 'cubic_cap',
       'distance_run_in_quarter', 'domain_cd', 'fin_yr_sale_amt', 'floor_area',
       'fuel', 'gcw', 'height', 'hp', 'imported_vch', 'ld_wt', 'length',
       'no_adv_units', 'no_cyl', 'no_of_trips', 'off_cd', 'op_dt',
       'other_criteria', 'pmt_catg', 'pmt_type', 'purchase_dt', 'rcpt_no',
       'regn_no', 'regn_type', 'route_class', 'route_length', 'sale_amt',
       'seat_cap', 'service_type', 'sleeper_cap', 'stand_cap', 'state_cd',
       'unld_wt', 'vch_catg', 'vch_purchase_as', 'vh_class', 'video_fitted',
       'wheelbase', 'width'],
      dtype='object')